#Directory, Libraries, and data

In [39]:
%cd /content/drive/MyDrive/Business Analyst course/Segmentation/RFM

/content/drive/MyDrive/Business Analyst course/Segmentation/RFM


In [40]:
#Libraries
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt

In [41]:
#Load the data
data=pd.read_csv("online_shoppers.csv", encoding="ISO-8859-1").dropna()
data.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom


#Data preparation

In [42]:
#Creating sales column
data['sales']=data['Quantity']*data['UnitPrice']
data.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.3


In [43]:
#Dates -Transform Date Variable
data['InvoiceDate']=pd.to_datetime(data['InvoiceDate'])

#Get last date available
snapshot_date=data['InvoiceDate'].max()+timedelta(days=1)
snapshot_date

Timestamp('2011-12-10 12:50:00')

In [44]:
#Aggregate on customer level
df=data.groupby(['CustomerID']).agg({
    'InvoiceDate':lambda x:(snapshot_date - x.max()).days,
    'InvoiceNo':'count',
    'sales':'sum'
})
df.head()

,InvoiceDate,InvoiceNo,sales
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


In [45]:
#Create basket/monetary variable
df['monetary']=df.sales/df.InvoiceNo
df.head(2)

,InvoiceDate,InvoiceNo,sales,monetary
CustomerID,,,,
12346.0,326,2,0.0,0.000000
12347.0,2,182,4310.0,23.681319


In [46]:
#Dropping sales variable
df=df.drop(columns="sales")
df.head(0)

,InvoiceDate,InvoiceNo,monetary
CustomerID,,,


In [48]:
#Changing variables' names
df.rename(columns={'InvoiceDate': 'recency',
                   'InvoiceNo':'frequency'},inplace=True)
df.head(5)

,recency,frequency,monetary
CustomerID,,,
12346.0,326,2,0.000000
12347.0,2,182,23.681319
12348.0,75,31,57.975484
12349.0,19,73,24.076027
12350.0,310,17,19.670588


#RFM Preparation

In [51]:
#Create Frequency groups
df['F']=pd.qcut(x=df['frequency'], q=4, labels=range(1,5,1))
df['M']=pd.qcut(x=df['monetary'],q=4,labels=range(1,5,1))
df['R']=pd.qcut(x=df['recency'],q=4,labels=range(4,0,-1))
df.head(3)

,recency,frequency,monetary,F,M,R
CustomerID,,,,,,
12346.0,326,2,0.000000,1,1,1
12347.0,2,182,23.681319,4,4,4
12348.0,75,31,57.975484,2,4,2


In [52]:
#RFM Score
df['RFM']=df[['R','F','M']].sum(axis=1)
df.head(3)

,recency,frequency,monetary,F,M,R,RFM
CustomerID,,,,,,,
12346.0,326,2,0.000000,1,1,1,3
12347.0,2,182,23.681319,4,4,4,12
12348.0,75,31,57.975484,2,4,2,8


In [53]:
#Create the RFM function
def rfm_segment(df):
  if df['RFM']>=11:
    return 'Superstar'
  elif((df['RFM']>=8)and(df['RFM']<11)):
    return 'Future Champion'
  elif((df['RFM']>=6)and (df['RFM']<8)):
    return 'High Potential'
  else:
    return 'Low Relevance'

#RFM


In [55]:
#Apply RFM function
df['RFM_level']=df.apply(rfm_segment, axis=1)
df.tail()

,recency,frequency,monetary,F,M,R,RFM,RFM_level
CustomerID,,,,,,,,
18280.0,278,10,18.060000,1,3,1,5,Low Relevance
18281.0,181,7,11.545714,1,2,1,4,Low Relevance
18282.0,8,13,13.584615,1,2,4,7,High Potential
18283.0,4,756,2.771005,4,1,4,9,Future Champion
18287.0,43,70,26.246857,3,4,3,10,Future Champion


In [62]:
#Looking into the segments
round(df.groupby('RFM_level').agg({
    'recency': 'mean',
    'frequency': 'mean',
    'monetary':['mean','count']
}),2)

recency frequency monetary      
                   mean      mean     mean count
RFM_level                                       
Future Champion   32.01    135.84    30.17  1768
High Potential   118.71     39.21    38.03  1477
Low Relevance    221.48     22.81    -0.59   753
Superstar          9.97    244.85    45.48   374